<h1 style="color:#D30982;">Quantum Widgets</h1>


What is in this notebook?

There are three widgets that I created for fun and for the aws blog post: https://aws.amazon.com/blogs/quantum-computing/bernoulli-line-and-the-bloch-sphere/


**`INSTRUCTIONS`**
1. The first code cell should run in the `default` python environment.
2. The second and third code cell will require the creation of a new environment because qutip has some breaking dependencies. So, I would suggest you to make a new environment on qBraid using environment manager and install qutip, numpy, amazon-braket-sdk and matplotlib.
3. The last code cell is just a rendering of a javascript bloch sphere made by https://github.com/JavaFXpert/



## Bernoulli Line

In [13]:

import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
from ipywidgets import Button, HBox, VBox

def plot_shots(p):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    langs = ['0', '1']
    shots = [int(np.round(p*100)),int(np.round((1-p)*100))]
    ax.bar(langs,shots)
    plt.show()

p=widgets.FloatSlider(min=0,max=1,value=0.5,step=0.0001,continuous=True, orientation='vertical',description=r'\(p_i\)')
out=widgets.interactive_output(plot_shots,{'p':p})
gui = HBox([p,out])
display(gui)


In [14]:
## WIDGET FOR BUILDING SINGLE QUBIT CIRCUITS WITH BRAKET
import sys
sys.path.append('../src/utils')
from braket.circuits import Circuit
from braket.devices import LocalSimulator
import qutip
import numpy as np
import ipyvuetify as v
from threading import Timer
import ipywidgets as widgets
import matplotlib.pyplot as plt
import copy
from qutip_bloch import Bloch
# from qutip import Bloch

class circuit_data():
    
    def __init__(self):
        self.result = None
        self.counts = None
        self.state = None
        self.value = Circuit().i(0)
        self.run = None
    def sim(self):
        device = LocalSimulator()
        # self.run = device.run(self.value,shots=100)
        # run circuit
        self.result = device.run(self.value,shots=100).result()
        # get measurement shots
        self.counts = self.result.measurement_counts
        # print(self.counts)
    def sim_statvector(self):
        circ1 = copy.deepcopy(self.value)
        circ1.state_vector()
        device = LocalSimulator()
        # run circuit
        self.result = device.run(circ1).result()
        # get measurement shots
        self.state = self.result.values[0]
        # print(self.counts)
        
        
circ_data = circuit_data()


# onClick listeners
def H(widget, event, data):
    circ_data.value=circ_data.value.h(0)
    H_button.value = not H_button.value
def X(widget, event, data):
    circ_data.value=circ_data.value.x(0)
    X_button.value = not X_button.value
def Y(widget, event, data):
    circ_data.value=circ_data.value.y(0)
    Y_button.value = not Y_button.value
def Z(widget, event, data):
    circ_data.value=circ_data.value.z(0)
    Z_button.value = not Z_button.value
def Rx(widget, event, data):
    circ_data.value=circ_data.value.rx(0, float(Rx_text.v_model)*np.pi/180)
    Rx_button.value = not Rx_button.value
def Ry(widget, event, data):
    circ_data.value=circ_data.value.ry(0, float(Ry_text.v_model)*np.pi/180)
    Z_button.value = not Z_button.value
def Rz(widget, event, data):
    circ_data.value=circ_data.value.rz(0, float(Rz_text.v_model)*np.pi/180)
    Rz_button.value = not Rz_button.value
def sim(widget, event, data):
    # circ_data.sim()
    sim_button.value = not sim_button.value
def plot(widget, event, data):
    # circ_data.sim()
    plot_button.value = not plot_button.value
def reset(widget, event, data):
    circ_data.value=Circuit().i(0)
    reset_button.value = not reset_button.value

# interactive output functions
def circ_interactive_output(H, X, Y, Z, Rx, Ry, Rz, reset):
    if reset:
        print(" ")
        return
    print(circ_data.value)

def bloch_sphere(sim,reset):
    if reset:
        print(" ")
        return
    b = qutip.Bloch()
    up = qutip.basis(2, 0)
    down = qutip.basis(2, 1)
    x = [1,0,0]
    y = [0,1,0]
    b.add_vectors(x)
    b.add_vectors(y)
    b.add_states(up)
    global circ_data
    if sim:
        sim_button.value = not sim_button.value
        circ_data.sim()
        circ_data.sim_statvector()
        # print(circ_data.result)
        final_state = (qutip.basis(2,0)*circ_data.state[0]+qutip.basis(2,1)*circ_data.state[1]).unit()
        b.add_states(final_state)
        b.render()
        b.show()
        
        # print(circ_data.counts)
def plot_results(plot,reset):
    if reset:
        reset_button.value = not reset_button.value
        print(" ")
        return
    if plot:
    # plot using Counter
        if circ_data.counts:
            plot_button.value = not plot_button.value
            plt.bar(circ_data.counts.keys(), circ_data.counts.values());
            plt.xlabel('bitstrings');
            plt.ylabel('counts');
        



# quantum_bit = q_NOT(q_NOT(quantum_bit))
# set up device
# defining layout for the buttons:
H_button = v.Btn(color='#9A11DA', dark=True, class_='ma-2', children=['H'], value = False)
X_button = v.Btn(color='#9A11DA', dark=True, class_='ma-2', children=['X'], value = False)
Y_button = v.Btn(color='#9A11DA', dark=True, class_='ma-2', children=['Y'], value = False)
Z_button = v.Btn(color='#9A11DA', dark=True, class_='ma-2', children=['Z'], value = False)
Rx_button = v.Btn(color='#9A11DA', dark=True, class_='ma-2', children=['Rx'], value = False)
Ry_button = v.Btn(color='#9A11DA', dark=True, class_='ma-2', children=['Ry'], value = False)
Rz_button = v.Btn(color='#9A11DA', dark=True, class_='ma-2', children=['Rz'], value = False)
sim_button = v.Btn(color='#D30982', dark=True, class_='ma-2', children=['Simulate'], value=False)
plot_button = v.Btn(color='#D30982', dark=True, class_='ma-2', children=['Plot Results'], value=False)
reset_button = v.Btn(color='red', dark=True, class_='ma-2', children=['Reset'], value=False)
Rx_text = v.TextField(style_="max-width:50px", v_model='0')
Ry_text = v.TextField(style_="max-width:50px", v_model='0')
Rz_text = v.TextField(style_="max-width:50px", v_model='0')


#onclick listeners for the buttons
H_button.on_event('click', H)
X_button.on_event('click', X)
Y_button.on_event('click', Y)
Z_button.on_event('click', Z)
Rx_button.on_event('click', Rx)
Ry_button.on_event('click', Ry)
Rz_button.on_event('click', Rz)
sim_button.on_event('click', sim)
plot_button.on_event('click', plot)
reset_button.on_event('click', reset)



# trigger dictionaries
circ_disp_triggers={"H":H_button,"X":X_button,"Y":Y_button,"Z":Z_button,"Rx":Rx_button,"Ry":Ry_button,"Rz":Rz_button,"reset":reset_button}
sim_triggers={"sim":sim_button,"reset":reset_button}
plot_trigger={"plot":plot_button,"reset":reset_button}

circ_disp = widgets.interactive_output(circ_interactive_output,circ_disp_triggers)
out=widgets.interactive_output(bloch_sphere, sim_triggers)
plot_disp = widgets.interactive_output(plot_results,plot_trigger)



# displaying the UI
button_layout = v.Layout(children=[H_button,X_button,Y_button,Z_button])
button_layout1 = v.Layout(children=[Rx_button, Rx_text,Ry_button,Ry_text, Rz_button,Rz_text])
layout = v.Layout(children=[sim_button, plot_button,reset_button])
output_layout = v.Layout(children=[out,plot_disp])

display(button_layout,button_layout1,layout,circ_disp,output_layout)



Layout(children=[Btn(children=['H'], class_='ma-2', color='#9A11DA', dark=True, layout=None, value=False), Btn…

Layout(children=[Btn(children=['Rx'], class_='ma-2', color='#9A11DA', dark=True, layout=None, value=False), Te…

Layout(children=[Btn(children=['Simulate'], class_='ma-2', color='#D30982', dark=True, layout=None, value=Fals…

Output()

Layout(children=[Output(), Output()], layout=None)

In [15]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import ipywidgets as widgets
import qutip
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox

def bloch_widget(psi,phi, npsi, nphi, vphi, vpsi, hide_axes):
    psi = psi*np.pi/180
    npsi = npsi*np.pi/180
    phi = phi*np.pi/180
    nphi = nphi*np.pi/180
    n = [np.sin(npsi)*np.cos(nphi), np.sin(npsi)*np.sin(nphi), np.cos(npsi)]
    n_matrix = np.sin(npsi)*np.cos(nphi)*qutip.sigmax()+np.sin(npsi)*np.sin(nphi)*qutip.sigmay()+np.cos(npsi)*qutip.sigmaz()
    n_1 = n_matrix.eigenstates()[1][0]
    n_0 = n_matrix.eigenstates()[1][1]
    b = Bloch()
    b.sphere_alpha = 0
    up = qutip.basis(2, 0)
    down = qutip.basis(2, 1)
    state = (qutip.basis(2,0)*np.cos(psi/2)+np.exp(1j*phi)*qutip.basis(2,1)*np.sin(psi/2)).unit()
    x = [1,0,0]
    y = [0,1,0]
    z = [0,0,1]
    overlap_vec = np.linalg.norm(n_0.overlap(state))*np.array(n)
    if not hide_axes:
        b.add_vectors(x,color='r')
        b.add_vectors(y,color='r')
        b.add_vectors(z,color='r')

    b.add_states(state, color='g')
    b.add_vectors(n,'b')
    # b.add_states(n_0,'b')
    b.add_vectors(overlap_vec,'g')
    b.view = [vpsi,vphi]
    b.figsize = [10,10]
    b.render()    
    b.show()
    
    
def plot_shots(psi,phi,npsi,nphi):
    psi = psi*np.pi/180
    phi = phi*np.pi/180
    npsi = npsi*np.pi/180
    nphi = nphi*np.pi/180
    state = (qutip.basis(2,0)*np.cos(psi/2)+np.exp(1j*phi)*qutip.basis(2,1)*np.sin(psi/2)).unit()
    n = [np.sin(npsi)*np.cos(nphi), np.sin(npsi)*np.sin(nphi), np.cos(npsi)]
    n_matrix = np.sin(npsi)*np.cos(nphi)*qutip.sigmax()+np.sin(npsi)*np.sin(nphi)*qutip.sigmay()+np.cos(npsi)*qutip.sigmaz()
    n_1 = n_matrix.eigenstates()[1][0]
    n_0 = n_matrix.eigenstates()[1][1]
    overlap_0 = np.linalg.norm(n_0.overlap(state))**2
    overlap_1 = np.linalg.norm(n_1.overlap(state))**2
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    langs = ['|0>', '|1>']
    students = [int(np.round(overlap_0*100)),int(np.round(overlap_1*100))]
    print(students)
    ax.bar(langs,students)
    plt.show()



hide_axes=widgets.Checkbox(value=False,description='Hide coordinate axes',disabled=False,indent=False)
psi=widgets.FloatSlider(min=0,max=360,step=1,value=0,description=r'\(\theta\)')
phi=widgets.FloatSlider(min=-180,max=180,value=0,step=1,description=r'\(\phi\)')
npsi=widgets.FloatSlider(min=0,max=180,step=1,value=0,description=r'\(\theta \) axis')
nphi=widgets.FloatSlider(min=0,max=360,value=0,step=1,description=r'\(\phi\) axis')
vpsi=widgets.FloatSlider(min=-180,max=180,step=1,value=-60,description=r'\(\theta \) azi')
vphi=widgets.FloatSlider(min=-180,max=180,value=35,step=1,description=r'\(\phi\) ele')

out=widgets.interactive_output(bloch_widget,{'psi':psi,'phi':phi,'npsi':npsi,'nphi':nphi,'vpsi':vpsi,'vphi':vphi, 'hide_axes': hide_axes})
out_shots=widgets.interactive_output(plot_shots,{'psi':psi,'phi':phi,'npsi':npsi,'nphi':nphi})
gui = HBox([out,out_shots])
out.layout.height='700px'
# display(widgets.VBox([psi,phi,npsi, nphi, vpsi,vphi]),gui)
display(widgets.HBox([widgets.VBox([psi,phi,npsi, nphi, vpsi,vphi, hide_axes,out_shots]),out]))



In [16]:

from IPython.display import IFrame
IFrame('https://javafxpert.github.io/grok-bloch/', width=950, height=700)
